In [1]:
from openai import OpenAI
import gradio as gr
import json

config=None
with open (".config.json",'r') as f:
    config=json.load(f)

def changeQuickQueryToFrom(choice):
    if choice=="FROM":
        return(
                gr.Dropdown(choices=["FROM","TO"],value="TO",interactive=False,show_label=False,scale=1,min_width=30,filterable=False)
            )
    
    return(
        gr.Dropdown(choices=["FROM","TO"],value="FROM",interactive=False,show_label=False,scale=1,min_width=30,filterable=False)
    )


def focusFreeQueryTextbox(text):
    if text==None or text=="":
        return(gr.Textbox(value="Show me a route from ",show_label=False,placeholder="Enter to submit.Please ask for a route"))
    return(gr.Textbox(value=text,show_label=False,placeholder="Enter to submit.Please ask for a route"))
def defocusFreeQueryTextbox(text):
    if text=="Show me a route from ":
        return(gr.Textbox(value=None,show_label=False,placeholder="Enter to submit.Please ask for a route"))
    return(gr.Textbox(value=text,show_label=False,placeholder="Enter to submit.Please ask for a route"))

def submitFreeQueryTextbox(text):
    client = OpenAI(api_key=config["OPENAI_API_KEY"])
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": 
            """
                You are a Singaporean Navigation Assistant.\n
                The user will ask you to search for a route from the nearest designated type of public service location to a specified location.\n
                You must search the public service location that matches user's query, then show a route from that public service location to the specified location.\n
                You will provide a route by providing start point name and end point name.\n
                Your response must strictly be in following format:[StartPointName#EndPointName]\n
            """
            },
            {"role": "user", "content": text}
        ]
    )
    returnText=str(completion.choices[0].message.content)[1:-1]
    returns=returnText.split("#")

    returns[0]=returns[0].replace(" ","+")
    returns[1]=returns[1].replace(" ","+")

    returns[0]="[\""+returns[0]+",+Singapore\","
    returns[1]="\""+returns[1]+",+Singapore\"]"
    returnText=returns[0]+returns[1]

    return(gr.HTML("<iframe src=http://fuqianshan.asuscomm.com:5173?body="+returnText+" width=100%% height=520px></iframe>"))

def submitQuickQueryTextbox(quickQueryToFrom,quickQueryDorpDown,quickQueryFromTo,quickQueryTextbox):
    text="Show me a route "+quickQueryToFrom+" the "+quickQueryDorpDown+" "+quickQueryFromTo+" "+quickQueryTextbox
    return(submitFreeQueryTextbox(text))

with gr.Blocks(title="BooleanPirates") as demo:
    with gr.Group():
        with gr.Tab("Quick Query"):
            with gr.Row():
                with gr.Row():
                    quickQueryToFrom=gr.Dropdown(choices=["TO","FROM"],value="TO",interactive=True,show_label=False,scale=1,min_width=60,filterable=False)
                    quickQueryDorpDown=gr.Dropdown(choices=[
                        "nearby Hospital",
                        "nearby Fire station",
                        "nearby Police station",
                        "nearest Bus interchange"
                    ],value="nearby Hospital",interactive=True,show_label=False,scale=2,min_width=100,filterable=False)
                with gr.Row():
                    quickQueryFromTo=gr.Dropdown(choices=["FROM","TO"],value="FROM",interactive=False,show_label=False,scale=1,min_width=60,filterable=False)
                    quickQueryTextbox=gr.Textbox(show_label=False,scale=2,min_width=100,placeholder="Type here&Enter")
        with gr.Tab("Free Query"):
            with gr.Row():
                freeQueryTextbox=gr.Textbox(show_label=False,placeholder="Enter to submit.Please ask for a route")

        googleMapHtmlBox=gr.HTML("<iframe src=http://fuqianshan.asuscomm.com:5173?body=[\"Fire+Station+Bukit+Merah,+Singapore\",\"SMU,+Singapore\"] width=100%% height=520px></iframe>")

        quickQueryToFrom.change(changeQuickQueryToFrom,quickQueryToFrom,quickQueryFromTo)
        quickQueryTextbox.submit(submitQuickQueryTextbox,[quickQueryToFrom,quickQueryDorpDown,quickQueryFromTo,quickQueryTextbox],googleMapHtmlBox)

        freeQueryTextbox.submit(submitFreeQueryTextbox,freeQueryTextbox,googleMapHtmlBox)
        freeQueryTextbox.focus(focusFreeQueryTextbox,freeQueryTextbox,freeQueryTextbox)
        freeQueryTextbox.blur(defocusFreeQueryTextbox,freeQueryTextbox,freeQueryTextbox)

demo.launch(server_name="0.0.0.0",server_port=7915)

#input()

/home/fustcianshan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://0.0.0.0:7915

To create a public link, set `share=True` in `launch()`.


In [ ]:
#test